# Notebook de Testes - NGROK & WhatsApp Bot

Este notebook é dedicado aos testes do sistema completo do WhatsApp Bot com ngrok tunnel.

## Objetivo dos Testes:
1. ✅ Verificar conectividade do ngrok
2. ✅ Testar recepção de webhooks
3. ✅ Validar processamento de mensagens de texto
4. 🔄 Testar processamento de áudio
5. 🔄 Testar processamento de imagens
6. 🔄 Validar resposta do bot via Evolution API

## Status do Sistema:
- **FastAPI Server**: ✅ Rodando
- **Ngrok Tunnel**: ✅ Ativo
- **Evolution API**: ✅ Conectado
- **OpenAI API**: ✅ Configurado
- **WhatsApp Test Number**: +55 62 9274-5972

In [ ]:
# Configuração inicial e importações
import os
import sys
import subprocess
import requests
import json
from datetime import datetime

# Adicionar o diretório src ao path
sys.path.append(os.path.join(os.getcwd(), 'src'))

print(f"🔧 Configuração inicial - {datetime.now().strftime('%H:%M:%S')}")
print(f"📁 Diretório atual: {os.getcwd()}")
print(f"🐍 Versão Python: {sys.version}")

# Verificar se .env existe
env_file = '.env'
if os.path.exists(env_file):
    print(f"✅ Arquivo .env encontrado")
else:
    print(f"❌ Arquivo .env não encontrado!")

: 

In [1]:
# Teste 1: Verificar status do ngrok
def verificar_ngrok_status():
    try:
        # Verificar se ngrok está rodando na porta padrão (4040)
        response = requests.get('http://localhost:4040/api/tunnels', timeout=5)
        
        if response.status_code == 200:
            tunnels = response.json()['tunnels']
            print(f"🟢 NGROK ATIVO - {len(tunnels)} túneis encontrados")
            
            for tunnel in tunnels:
                public_url = tunnel['public_url']
                local_addr = tunnel['config']['addr']
                print(f"   🌐 URL Pública: {public_url}")
                print(f"   🏠 URL Local: {local_addr}")
                
                # Testar conectividade da URL pública
                try:
                    test_response = requests.get(f"{public_url}/health", timeout=10)
                    if test_response.status_code == 200:
                        print(f"   ✅ Conectividade OK - Health check passou")
                    else:
                        print(f"   ⚠️ Health check retornou: {test_response.status_code}")
                except Exception as e:
                    print(f"   ❌ Erro na conectividade: {str(e)}")
            
            return tunnels
        else:
            print(f"❌ NGROK não está respondendo - Status: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"❌ NGROK não está rodando ou não acessível: {str(e)}")
        return None

# Executar teste
print("=" * 50)
print("🧪 TESTE 1: STATUS DO NGROK")
print("=" * 50)
tunnels = verificar_ngrok_status()

🧪 TESTE 1: STATUS DO NGROK
❌ NGROK não está rodando ou não acessível: name 'requests' is not defined


In [ ]:
# Teste 2: Verificar servidor FastAPI local
def verificar_fastapi_local():
    try:
        # Testar o servidor local FastAPI
        response = requests.get('http://localhost:8000/health', timeout=5)
        
        if response.status_code == 200:
            data = response.json()
            print(f"🟢 FASTAPI LOCAL ATIVO")
            print(f"   📊 Status: {data.get('status', 'N/A')}")
            print(f"   ⏰ Timestamp: {data.get('timestamp', 'N/A')}")
            return True
        else:
            print(f"⚠️ FastAPI respondeu com status: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"❌ FASTAPI LOCAL não está acessível: {str(e)}")
        return False

# Executar teste
print("=" * 50)
print("🧪 TESTE 2: SERVIDOR FASTAPI LOCAL")
print("=" * 50)
fastapi_ok = verificar_fastapi_local()

In [ ]:
# Teste 3: Simular webhook do Evolution API
def simular_webhook_texto():
    """Simula o envio de uma mensagem de texto via webhook"""
    
    if not tunnels:
        print("❌ Ngrok não está ativo - não é possível testar webhook")
        return False
    
    # Pegar a URL pública do ngrok
    public_url = tunnels[0]['public_url']
    webhook_url = f"{public_url}/webhook/whatsapp"
    
    # Dados simulados de uma mensagem de texto do Evolution API
    webhook_data = {
        "event": "messages.upsert",
        "instance": "centro-oeste-drywall",
        "data": {
            "key": {
                "remoteJid": "556299999999@s.whatsapp.net",
                "fromMe": False,
                "id": f"test_message_{datetime.now().strftime('%Y%m%d%H%M%S')}"
            },
            "messageType": "textMessage",
            "message": {
                "conversation": "Olá, este é um teste do notebook!"
            },
            "source": "web",
            "messageTimestamp": int(datetime.now().timestamp())
        }
    }
    
    try:
        print(f"🚀 Enviando webhook para: {webhook_url}")
        print(f"📨 Dados da mensagem: {webhook_data['data']['message']['conversation']}")
        
        response = requests.post(
            webhook_url,
            json=webhook_data,
            headers={"Content-Type": "application/json"},
            timeout=10
        )
        
        print(f"📡 Status da resposta: {response.status_code}")
        
        if response.status_code == 200:
            print(f"✅ WEBHOOK PROCESSADO COM SUCESSO")
            try:
                response_data = response.json()
                print(f"📋 Resposta: {json.dumps(response_data, indent=2)}")
            except:
                print(f"📋 Resposta (texto): {response.text}")
            return True
        else:
            print(f"❌ Webhook falhou - Status: {response.status_code}")
            print(f"📋 Resposta: {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Erro ao enviar webhook: {str(e)}")
        return False

# Executar teste
print("=" * 50)
print("🧪 TESTE 3: SIMULAÇÃO DE WEBHOOK (TEXTO)")
print("=" * 50)
webhook_ok = simular_webhook_texto()

In [ ]:
# Teste 4: Testar Evolution API - Enviar mensagem de resposta
def testar_envio_resposta():
    """Testa o envio de uma mensagem de resposta via Evolution API"""
    
    try:
        from config.settings import Settings
        
        # Carregar configurações
        settings = Settings()
        
        # URL da Evolution API para enviar mensagem
        url = f"{settings.EVOLUTION_API_URL}/message/sendText/{settings.EVOLUTION_INSTANCE_NAME}"
        
        # Dados para envio de mensagem
        message_data = {
            "number": "556292745972",  # Número de teste
            "text": f"🤖 Teste de resposta automática - {datetime.now().strftime('%H:%M:%S')}"
        }
        
        headers = {
            "Content-Type": "application/json",
            "apikey": settings.EVOLUTION_API_KEY
        }
        
        print(f"📱 Enviando mensagem para: {message_data['number']}")
        print(f"💬 Mensagem: {message_data['text']}")
        print(f"🌐 URL: {url}")
        
        response = requests.post(
            url,
            json=message_data,
            headers=headers,
            timeout=10
        )
        
        print(f"📡 Status da resposta: {response.status_code}")
        
        if response.status_code == 201:
            print(f"✅ MENSAGEM ENVIADA COM SUCESSO")
            try:
                response_data = response.json()
                print(f"📋 Resposta da API: {json.dumps(response_data, indent=2)}")
            except:
                print(f"📋 Resposta (texto): {response.text}")
            return True
        else:
            print(f"❌ Falha no envio - Status: {response.status_code}")
            print(f"📋 Resposta: {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Erro ao enviar mensagem: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

# Executar teste
print("=" * 50)
print("🧪 TESTE 4: ENVIO DE MENSAGEM VIA EVOLUTION API")
print("=" * 50)
envio_ok = testar_envio_resposta()

In [ ]:
# Teste 5: Testar processamento de áudio (simulado)
def simular_webhook_audio():
    """Simula o recebimento de uma mensagem de áudio"""
    
    if not tunnels:
        print("❌ Ngrok não está ativo - não é possível testar webhook de áudio")
        return False
    
    # Pegar a URL pública do ngrok
    public_url = tunnels[0]['public_url']
    webhook_url = f"{public_url}/webhook/whatsapp"
    
    # Dados simulados de uma mensagem de áudio do Evolution API
    webhook_data = {
        "event": "messages.upsert",
        "instance": "centro-oeste-drywall",
        "data": {
            "key": {
                "remoteJid": "556299999999@s.whatsapp.net",
                "fromMe": False,
                "id": f"test_audio_{datetime.now().strftime('%Y%m%d%H%M%S')}"
            },
            "messageType": "audioMessage",
            "message": {
                "audioMessage": {
                    "url": "https://evolution-api.com/files/instances/centro-oeste-drywall/messages/test_audio.ogg",
                    "mimetype": "audio/ogg; codecs=opus",
                    "seconds": 5
                }
            },
            "source": "web",
            "messageTimestamp": int(datetime.now().timestamp())
        }
    }
    
    try:
        print(f"🎵 Enviando webhook de áudio para: {webhook_url}")
        print(f"📨 URL do áudio: {webhook_data['data']['message']['audioMessage']['url']}")
        
        response = requests.post(
            webhook_url,
            json=webhook_data,
            headers={"Content-Type": "application/json"},
            timeout=15
        )
        
        print(f"📡 Status da resposta: {response.status_code}")
        
        if response.status_code == 200:
            print(f"✅ WEBHOOK DE ÁUDIO PROCESSADO")
            try:
                response_data = response.json()
                print(f"📋 Resposta: {json.dumps(response_data, indent=2)}")
            except:
                print(f"📋 Resposta (texto): {response.text}")
            return True
        else:
            print(f"❌ Webhook de áudio falhou - Status: {response.status_code}")
            print(f"📋 Resposta: {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Erro ao enviar webhook de áudio: {str(e)}")
        return False

# Executar teste
print("=" * 50)
print("🧪 TESTE 5: SIMULAÇÃO DE WEBHOOK (ÁUDIO)")
print("=" * 50)
audio_ok = simular_webhook_audio()

In [ ]:
# Teste 6: Resumo completo dos testes
def gerar_relatorio_final():
    """Gera um relatório final com todos os resultados dos testes"""
    
    print("=" * 60)
    print("📊 RELATÓRIO FINAL DOS TESTES NGROK")
    print("=" * 60)
    
    resultados = {
        "Ngrok Tunnel": "✅ ATIVO" if tunnels else "❌ INATIVO",
        "FastAPI Local": "✅ OK" if 'fastapi_ok' in globals() and fastapi_ok else "❌ FALHA",
        "Webhook Texto": "✅ OK" if 'webhook_ok' in globals() and webhook_ok else "❌ FALHA",
        "Envio Mensagem": "✅ OK" if 'envio_ok' in globals() and envio_ok else "❌ FALHA", 
        "Webhook Áudio": "✅ OK" if 'audio_ok' in globals() and audio_ok else "❌ FALHA"
    }
    
    for teste, status in resultados.items():
        print(f"   {teste:.<20} {status}")
    
    print("\n" + "=" * 60)
    
    # Contar sucessos
    sucessos = sum(1 for status in resultados.values() if "✅" in status)
    total = len(resultados)
    
    print(f"🎯 TAXA DE SUCESSO: {sucessos}/{total} ({(sucessos/total)*100:.1f}%)")
    
    if sucessos == total:
        print("🎉 TODOS OS TESTES PASSARAM! Sistema está 100% operacional!")
    elif sucessos >= total * 0.8:
        print("⚠️ Sistema majoritariamente funcional - verificar falhas menores")
    else:
        print("❌ Sistema com problemas significativos - revisar configurações")
    
    print("\n📱 Para testar com mensagens reais:")
    print(f"   • Envie mensagem para: +55 62 9274-5972")
    print(f"   • URL do Webhook: {tunnels[0]['public_url']}/webhook/whatsapp" if tunnels else "   • Ngrok não está ativo")
    
    return resultados

# Executar relatório final
relatorio = gerar_relatorio_final()

## 🔧 Comandos Úteis para Troubleshooting

### Reiniciar Ngrok:
```bash
# Parar ngrok atual
pkill -f ngrok

# Iniciar novo tunnel
ngrok http 8000 --authtoken=2pY1zXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
```

### Verificar logs do FastAPI:
```bash
# Se rodando em terminal separado, verificar logs
# Ou reiniciar com:
uvicorn src.main:app --reload --host 0.0.0.0 --port 8000
```

### Testar manualmente via curl:
```bash
# Testar health check
curl http://localhost:8000/health

# Testar webhook (substitua URL_NGROK pela URL atual)
curl -X POST URL_NGROK/webhook/whatsapp \
  -H "Content-Type: application/json" \
  -d '{"event":"messages.upsert","instance":"centro-oeste-drywall","data":{"messageType":"textMessage","message":{"conversation":"teste"}}}'
```

### Verificar configurações:
- ✅ Arquivo `.env` com todas as chaves de API
- ✅ Ngrok configurado com authtoken válido
- ✅ Evolution API acessível e instance ativa
- ✅ OpenAI API key válida
- ✅ Supabase configurado corretamente